In [382]:
import numpy as np
import pandas as pd
import google
from google.cloud import bigquery
from google.oauth2 import service_account
import time
from sklearn.model_selection import train_test_split
import xgboost as xgb
xgb_regressor = xgb.XGBRegressor()
import random
import os
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
import pandas as pd
from datetime import date, timedelta

#import lightgbm as lgb
#!pip3 install google-cloud-bigquery
#!pip3 install xgboost
#!pip3 install pandas-gbq
#!pip3 install db-dtypes


### Questions and notes 

In [383]:
# How are we deciding what is train and test split
# Some combos have less show sale amount as 1, re check them.

## Read data bigquery

In [509]:
start = time.time()

import os

key_path="vcm-dwh-4b93f2172cf5.json"
credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/bigquery"],
)


def client_query(key_path):
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/bigquery"],
        )
    # Construct a BigQuery client object.
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    query = """
  
SELECT 
EXTRACT(YEAR FROM A.calday) AS YEAR,
EXTRACT(WEEK FROM A.calday) AS WEEK,
date_trunc(A.calday, week(monday)) calday,
A.STORE_ID,
  SUM(A.SALE_AMOUNT - A.TAX_AMOUNT +
      IF
        (DISCOUNT_TYPE IN ('ZRTA',
            'ZCRT'), DISCOUNT_VALUE, 0)) AS SALE_AMOUNT FROM `vcm-dwh.WCM_DWH_PRD_SHARING.F_SALE_PROD_VW` A
LEFT JOIN WCM_DWH_PRD_SHARING.D_STORE_VW D
ON A.STORE_ID = D.STORE_ID
LEFT JOIN WCM_DWH_PRD_SHARING.D_PRODUCT_VW E
ON A.PRODUCT_ID = E.PRODUCT_ID
WHERE A.CALDAY between '2021-01-01' and '2023-12-03'
AND D.BUSINESS_UNIT IN ('1500','2000')
AND E.MCH2_ID IN ('101','102','201','202','203')
AND SALE_QTY > 0 
group by 1,2,3,4
    """
  



    query_job = client.query(query)  # Make an API request.
    df = pd.read_gbq(query, credentials=credentials)
    return df
# Business unit and mch2 levels are already applied

df = client_query(key_path)
#print(df.calday.head())

end = time.time()
print(end)

1702535978.1158242


In [524]:
df.shape

(902909, 5)

In [510]:
df.head()

,YEAR,WEEK,calday,STORE_ID,SALE_AMOUNT
0,2023,3,2023-01-16,2123,128182966.0
1,2023,3,2023-01-16,5696,230923363.0
2,2023,3,2023-01-16,6213,186992899.0
3,2023,3,2023-01-16,5209,150655345.0
4,2023,3,2023-01-16,6229,116721688.0


In [511]:
df.calday.max()

datetime.date(2023, 11, 27)

In [512]:
df_combined['STORE_ID'].nunique()

3936

In [513]:
df['calday']= pd.to_datetime(df['calday'])
df.dtypes


YEAR                    Int64
WEEK                    Int64
calday         datetime64[ns]
STORE_ID               object
SALE_AMOUNT           float64
dtype: object

#### Add rows for Oct-Dec

In [514]:
# def get_monday(date):
#     return date - timedelta(days=date.weekday())

# Generating additional data for October, November, and December
#additional_data_oct = [get_monday(date(2023, 10, i)) for i in range(1, 32)]
#additional_data_nov = [get_monday(date(2023, 11, i)) for i in range(1, 31)]
# additional_data_dec = [get_monday(date(2023, 12, i)) for i in range(1, 32)]


# #additional_data = additional_data_oct + additional_data_nov + additional_data_dec
# df_additional = pd.DataFrame(additional_data_dec, columns=['calday'])

# df_additional['calday']= pd.to_datetime(df_additional['calday'])

# df_additional['YEAR'] = df_additional['calday'].dt.year
# df_additional['WEEK'] = df_additional['calday'].dt.isocalendar().week

# Concatenating the original DataFrame and the DataFrame with additional data
#result_df = pd.concat([df, df_additional], ignore_index=True)

#### Add rows for Dec

In [515]:
import pandas as pd
from datetime import datetime, timedelta

# Specify the year and month
year = 2023
month = 12


STORE_ID = df["STORE_ID"].unique().tolist()  # Add your actual mch_store values

#STORE_ID = ['1518']


# Create an empty DataFrame to store the results
df_combined = pd.DataFrame()

for STORE_ID in STORE_ID:
    # Get the first day of the month
    first_day = datetime(year, month, 1)

    # Find the first Monday of the month
    first_monday = first_day + timedelta(days=(6 - first_day.weekday() + 1) % 7)

    # Calculate all Mondays for December
    mondays = [first_monday + timedelta(days=(i * 7)) for i in range((31 - first_monday.day) // 7 + 1)]

    # Create a DataFrame for the current mch_store
    df_additional = pd.DataFrame({'STORE_ID': [STORE_ID] * len(mondays), 'calday': mondays})

    # Concatenate the current DataFrame with the overall DataFrame
    df_combined = pd.concat([df_combined, df_additional], ignore_index=True)

In [516]:
df_combined['STORE_ID'].nunique()

3936

In [517]:

df_combined = df_combined.dropna(subset=['STORE_ID'])
df_combined

,STORE_ID,calday
0,2123,2023-12-04
1,2123,2023-12-11
2,2123,2023-12-18
3,2123,2023-12-25
4,5696,2023-12-04
...,...,...
15739,2AN7,2023-12-25
15740,6A01,2023-12-04
15741,6A01,2023-12-11
15742,6A01,2023-12-18


In [525]:
df_back = pd.concat([df, df_combined])

In [526]:
df_back.reset_index(inplace=True,drop=True)

In [527]:
df_back.shape

(918653, 5)

In [528]:
df_back.tail()

,YEAR,WEEK,calday,STORE_ID,SALE_AMOUNT
918648,<NA>,<NA>,2023-12-25,2AN7,NaN
918649,<NA>,<NA>,2023-12-04,6A01,NaN
918650,<NA>,<NA>,2023-12-11,6A01,NaN
918651,<NA>,<NA>,2023-12-18,6A01,NaN
918652,<NA>,<NA>,2023-12-25,6A01,NaN


In [529]:
df_back = df_back.rename(columns={'calday': 'CALDAY'})

df_back.head()

,YEAR,WEEK,CALDAY,STORE_ID,SALE_AMOUNT
0,2023,3,2023-01-16,2123,128182966.0
1,2023,3,2023-01-16,5696,230923363.0
2,2023,3,2023-01-16,6213,186992899.0
3,2023,3,2023-01-16,5209,150655345.0
4,2023,3,2023-01-16,6229,116721688.0


In [530]:
df_back.tail()

,YEAR,WEEK,CALDAY,STORE_ID,SALE_AMOUNT
918648,<NA>,<NA>,2023-12-25,2AN7,NaN
918649,<NA>,<NA>,2023-12-04,6A01,NaN
918650,<NA>,<NA>,2023-12-11,6A01,NaN
918651,<NA>,<NA>,2023-12-18,6A01,NaN
918652,<NA>,<NA>,2023-12-25,6A01,NaN


## Store level features

#### Sales min , max , std  feature %

In [531]:
### can take an average of dicount% at mch3 level
## For every product_store combo, take brand_store combo
### gdp and cpi index at month quarter level
### Finally taking 20 festures? All

In [553]:
# Rolling up the table at modeling level
#df_back.fillna(1,inplace=True)

df1 = df_back.groupby(['STORE_ID', 'CALDAY']).agg(
    min_sale=('SALE_AMOUNT', np.min),
    max_sale=('SALE_AMOUNT', np.max),
    std_sale=('SALE_AMOUNT', np.std),
    agg_sale=('SALE_AMOUNT', np.sum)).reset_index()

df1.head()

,STORE_ID,CALDAY,min_sale,max_sale,std_sale,agg_sale
0,1511,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08
1,1511,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09
2,1511,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09
3,1511,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09
4,1511,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09


In [554]:
#df1 = df
df1.shape

(469433, 6)

#### Sales growth 

In [555]:
order = 8  #number of weeks
order1 = 'lag_' + str(order)

In [556]:
df1.sort_values(['STORE_ID','CALDAY'],inplace=True)
df1[order1] = df1.groupby(['STORE_ID'])['agg_sale'].shift(order)
df1['sales_growth'] = (df1['agg_sale']/df1[order1] - 1)*100
df1.head()

,STORE_ID,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth
0,1511,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN
1,1511,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN
2,1511,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN
3,1511,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN
4,1511,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN


In [557]:
df1['mch_store'] = df1['STORE_ID']
df1=df1.drop(['STORE_ID'], axis=1)
df1.head(10)

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store
0,2020-12-28,1.678343e+08,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511
1,2021-01-04,1.885254e+08,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511
2,2021-01-11,2.469085e+08,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511
3,2021-01-18,2.320193e+08,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511
4,2021-01-25,3.155683e+08,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511
5,2021-02-01,3.411797e+08,1.720520e+09,9.753411e+08,2.061700e+09,NaN,NaN,1511
6,2021-02-08,1.162177e+09,1.162177e+09,NaN,1.162177e+09,NaN,NaN,1511
7,2021-02-15,1.107816e+08,5.064076e+08,2.797498e+08,6.171892e+08,NaN,NaN,1511
8,2021-02-22,1.642332e+08,6.193793e+08,3.218369e+08,7.836125e+08,6.019433e+08,30.180442,1511
9,2021-03-01,1.833503e+08,6.429289e+08,3.249711e+08,8.262792e+08,1.070781e+09,-22.833961,1511


In [558]:
df1['agg_sale'].min()

0.0

In [559]:
df1[df1['mch_store']=='1535']

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store
2189,2020-12-28,1.373803e+09,2.886295e+09,1.069493e+09,4.260098e+09,NaN,NaN,1535
2190,2021-01-04,1.810112e+09,5.682878e+09,2.738459e+09,7.492990e+09,NaN,NaN,1535
2191,2021-01-11,1.968053e+09,6.578381e+09,3.259995e+09,8.546434e+09,NaN,NaN,1535
2192,2021-01-18,2.180273e+09,6.748100e+09,3.229942e+09,8.928373e+09,NaN,NaN,1535
2193,2021-01-25,1.709841e+09,7.670190e+09,4.214603e+09,9.380031e+09,NaN,NaN,1535
...,...,...,...,...,...,...,...,...
2341,2023-11-27,1.196566e+09,3.816343e+09,1.852462e+09,5.012909e+09,5.588775e+09,-10.303979,1535
2342,2023-12-04,NaN,NaN,NaN,0.000000e+00,5.390591e+09,-100.000000,1535
2343,2023-12-11,NaN,NaN,NaN,0.000000e+00,6.243867e+09,-100.000000,1535
2344,2023-12-18,NaN,NaN,NaN,0.000000e+00,5.341575e+09,-100.000000,1535


### Store and brand combo

#### Lag features

In [560]:
## Change range here

In [561]:
def sales_lag(df):
    for i in range(order,8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df["lag_" + str(i)] = df.groupby(['mch_store'])['agg_sale'].shift(i)
    return(df)

def  min_sales_lag(df):
    for i in range(order,8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df["minslag_" + str(i)] = df.groupby(['mch_store'])['min_sale'].shift(i)
    return(df)

def  max_sales_lag(df):
    for i in range(order,8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df["maxslag_" + str(i)] = df.groupby(['mch_store'])['max_sale'].shift(i)
    return(df)

def  std_sales_lag(df):
    for i in range(order,8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df["stdslag_" + str(i)] = df.groupby(['mch_store'])['std_sale'].shift(i)
    return(df)

def growth_lag(df):
    for i in range(order,8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df["sglag_" + str(i)] = df.groupby(['mch_store'])['sales_growth'].shift(i)
    return(df)

#### Creating rolling window features

In [562]:
def sales_rolling(df):
    for i in range(8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df['rolling_mean_'+ str(i)] = df.groupby(['mch_store'])[order1].rolling(window=i).mean().reset_index()[order1]
        df['rolling_std_'+ str(i)] = df.groupby(['mch_store'])[order1].rolling(window=i).std().reset_index()[order1]
        df['rolling_min_'+ str(i)] = df.groupby(['mch_store'])[order1].rolling(window=i).min().reset_index()[order1]
        df['rolling_max_'+ str(i)] = df.groupby(['mch_store'])[order1].rolling(window=i).max().reset_index()[order1]

    return(df)

#### Creating ewma features

In [563]:
def sales_ewma(df):
    for i in range(2,10):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df['ewma_mean_'+ str(i)] = df.groupby(['mch_store'])[order1].ewm(span=i).mean().reset_index()[order1]
        df['ewma_std_'+ str(i)] = df.groupby(['mch_store'])[order1].ewm(span=i).std().reset_index()[order1]
    return(df)

#### Creating Expanding window features


In [564]:
def sales_expanding(df):
    for i in range(8):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df['expanding_mean_'+ str(i)] = df.groupby(['mch_store'])[order1].expanding(i).mean().reset_index()[order1]
        df['expanding_std_'+ str(i)] = df.groupby(['mch_store'])[order1].expanding(i).std().reset_index()[order1]
        df['expanding_std_'+ str(i)] = df.groupby(['mch_store'])[order1].expanding(i).std().reset_index()[order1]
        df['expanding_min_'+ str(i)] = df.groupby(['mch_store'])[order1].expanding(i).min().reset_index()[order1]
        df['expanding_max_'+ str(i)] = df.groupby(['mch_store'])[order1].expanding(i).max().reset_index()[order1]

    return(df)

In [565]:
sales_lag(df1)
min_sales_lag(df1)
max_sales_lag(df1)
std_sales_lag(df1)
growth_lag(df1)
sales_expanding(df1)
sales_ewma(df1)
sales_rolling(df1)


#df1.drop(['Discount_Perc','sales_growth',order1],axis=1,inplace=True)

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,rolling_min_5,rolling_max_5,rolling_mean_6,rolling_std_6,rolling_min_6,rolling_max_6,rolling_mean_7,rolling_std_7,rolling_min_7,rolling_max_7
0,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469428,2023-11-20,64859258.0,6.485926e+07,NaN,6.485926e+07,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469429,2023-12-04,NaN,NaN,NaN,0.000000e+00,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469430,2023-12-11,NaN,NaN,NaN,0.000000e+00,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469431,2023-12-18,NaN,NaN,NaN,0.000000e+00,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [566]:
df1['CALDAY']= pd.to_datetime(df1['CALDAY'])
df1.shape

(469433, 88)

In [567]:
data_final = df1.copy()

In [568]:
data_final['agg_sale'].min()

0.0

In [569]:
## Add GDP and CPI index numbers 

### GDP Growth Rate

In [570]:
data = {'GDP_quarter': ['2021-01-01','2021-04-01','2021-07-01','2021-10-01',
                        '2022-01-01','2022-04-01','2022-07-01','2022-10-01',
                        '2023-01-01','2023-04-01','2023-07-01','2023-10-01'
                       ], 
        'GDP_Growth': [4.48, 4.72,6.73, -6.02,
                       5.22, 5.05, 7.83, 13.71, 
                       5.92, 2.28, 4.14,5.33]}  
 
GDP_Data = pd.DataFrame(data)  
GDP_Data = GDP_Data.sort_values(['GDP_quarter'], ascending=[True])

for lag in [1,2,3,4]:  # approximate days for 3, 6, 9 and 12 months
    GDP_Data[f'gdp_lag_{lag}'] = GDP_Data['GDP_Growth'].shift(lag)

GDP_Data = GDP_Data.fillna(0)    


    
GDP_Data['perc_change_lag1'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_1'])/GDP_Data['gdp_lag_1']
GDP_Data['perc_change_lag2'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_2'])/GDP_Data['gdp_lag_2']
GDP_Data['perc_change_lag3'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_3'])/GDP_Data['gdp_lag_3']
GDP_Data['perc_change_lag4'] =  GDP_Data['GDP_Growth']- (GDP_Data['gdp_lag_4'])/GDP_Data['gdp_lag_4']

GDP_Data['perc_change_lag12'] =  (GDP_Data['gdp_lag_1']-GDP_Data['gdp_lag_2'])/GDP_Data['gdp_lag_2']
GDP_Data['perc_change_lag23'] =  (GDP_Data['gdp_lag_2']-GDP_Data['gdp_lag_3'])/GDP_Data['gdp_lag_3']
GDP_Data['perc_change_lag34'] =  (GDP_Data['gdp_lag_3']-GDP_Data['gdp_lag_4'])/GDP_Data['gdp_lag_4']


cols = ["perc_change_lag1","perc_change_lag2","perc_change_lag3","perc_change_lag4",
        "perc_change_lag12","perc_change_lag23","perc_change_lag34"]

for i in range(0,len(cols)):
    GDP_Data[cols[i]]= GDP_Data[cols[i]].fillna(0)
    GDP_Data[cols[i]]= GDP_Data[cols[i]].replace([np.inf, -np.inf], 0)
    
    
GDP_Data

,GDP_quarter,GDP_Growth,gdp_lag_1,gdp_lag_2,gdp_lag_3,gdp_lag_4,perc_change_lag1,perc_change_lag2,perc_change_lag3,perc_change_lag4,perc_change_lag12,perc_change_lag23,perc_change_lag34
0,2021-01-01,4.48,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000
1,2021-04-01,4.72,4.48,0.00,0.00,0.00,3.72,0.00,0.00,0.00,0.000000,0.000000,0.000000
2,2021-07-01,6.73,4.72,4.48,0.00,0.00,5.73,5.73,0.00,0.00,0.053571,0.000000,0.000000
3,2021-10-01,-6.02,6.73,4.72,4.48,0.00,-7.02,-7.02,-7.02,0.00,0.425847,0.053571,0.000000
4,2022-01-01,5.22,-6.02,6.73,4.72,4.48,4.22,4.22,4.22,4.22,-1.894502,0.425847,0.053571
5,2022-04-01,5.05,5.22,-6.02,6.73,4.72,4.05,4.05,4.05,4.05,-1.867110,-1.894502,0.425847
6,2022-07-01,7.83,5.05,5.22,-6.02,6.73,6.83,6.83,6.83,6.83,-0.032567,-1.867110,-1.894502
7,2022-10-01,13.71,7.83,5.05,5.22,-6.02,12.71,12.71,12.71,12.71,0.550495,-0.032567,-1.867110
8,2023-01-01,5.92,13.71,7.83,5.05,5.22,4.92,4.92,4.92,4.92,0.750958,0.550495,-0.032567
9,2023-04-01,2.28,5.92,13.71,7.83,5.05,1.28,1.28,1.28,1.28,-0.568198,0.750958,0.550495


In [571]:
data_final['calday_quarter'] = pd.PeriodIndex(data_final['CALDAY'], freq='q')
GDP_Data['gdp_quarter_1'] = pd.PeriodIndex(GDP_Data.GDP_quarter, freq='q')
data_final = data_final.merge(GDP_Data, left_on ='calday_quarter',right_on='gdp_quarter_1', how='left')

col_drop =["gdp_quarter_1","calday_quarter","GDP_quarter"]

data_final=data_final.drop(col_drop, axis=1)
data_final.head()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,gdp_lag_2,gdp_lag_3,gdp_lag_4,perc_change_lag1,perc_change_lag2,perc_change_lag3,perc_change_lag4,perc_change_lag12,perc_change_lag23,perc_change_lag34
0,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### CPI data

In [572]:
CPI_data = {'CPI_month': ['2021-01-01','2021-02-01','2021-03-01','2021-04-01',
                          '2021-05-01','2021-06-01','2021-07-01','2021-08-01',
                          '2021-09-01','2021-10-01','2021-11-01','2021-12-01',
                          '2022-01-01','2022-02-01','2022-03-01','2022-04-01',
                          '2022-05-01','2022-06-01','2022-07-01','2022-08-01',
                          '2022-09-01','2022-10-01','2022-11-01','2022-12-01',
                          '2023-01-01','2023-02-01','2023-03-01','2023-04-01',
                          '2023-05-01','2023-06-01','2023-07-01','2023-08-01',
                          '2023-09-01','2023-10-01','2023-11-01'], 
            'CPI_index': [103.27, 104.84, 104.56 , 104.52,
                          104.68, 104.86, 105.53 , 105.79,
                          105.14, 104.93, 105.27 , 105.08,
                          105.28,106.33,107.08,107.28,
                          107.68,108.42,108.85,108.86,
                          109.29,109.44,109.87,109.95,
                          110.42,110.92,110.67,110.29,
                          110.3,110.59,111.09,112.07,
                          113.28,113.38,113.66]}
CPI_Df = pd.DataFrame(CPI_data)
CPI_Df = CPI_Df.sort_values(['CPI_index'], ascending=[True])

for lag in [1,2,3,4]:
    CPI_Df[f'cpi_lag_{lag}'] = CPI_Df['CPI_index'].shift(lag)

CPI_Df = CPI_Df.fillna(0)


CPI_Df['cpi_perc_change_lag1'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_1'])/CPI_Df['cpi_lag_1']
CPI_Df['cpi_perc_change_lag2'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_2'])/CPI_Df['cpi_lag_2']
CPI_Df['cpi_perc_change_lag3'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_3'])/CPI_Df['cpi_lag_3']
CPI_Df['cpi_perc_change_lag4'] = CPI_Df['CPI_index']- (CPI_Df['cpi_lag_4'])/CPI_Df['cpi_lag_4']

CPI_Df['cpi_perc_change_lag12'] = (CPI_Df['cpi_lag_1']-CPI_Df['cpi_lag_2'])/CPI_Df['cpi_lag_2']
CPI_Df['cpi_perc_change_lag23'] = (CPI_Df['cpi_lag_2']-CPI_Df['cpi_lag_3'])/CPI_Df['cpi_lag_3']
CPI_Df['cpi_perc_change_lag34'] = (CPI_Df['cpi_lag_3']-CPI_Df['cpi_lag_4'])/CPI_Df['cpi_lag_4']



cols = ["cpi_perc_change_lag1","cpi_perc_change_lag2","cpi_perc_change_lag3","cpi_perc_change_lag4",
        "cpi_perc_change_lag12","cpi_perc_change_lag23","cpi_perc_change_lag34"]

for i in range(0,len(cols)):
    CPI_Df[cols[i]]= CPI_Df[cols[i]].fillna(0)
    CPI_Df[cols[i]]= CPI_Df[cols[i]].replace([np.inf, -np.inf], 0)


data_final['data_month'] = (data_final['CALDAY'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))


data_final['data_month']= pd.to_datetime(data_final['data_month'])
CPI_Df['CPI_month']= pd.to_datetime(CPI_Df['CPI_month'])


data_final = data_final.merge(CPI_Df, left_on ='data_month',right_on='CPI_month', how='left')



col_drop =["data_month","CPI_month"]

data_final=data_final.drop(col_drop, axis=1)
data_final.head()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_2,cpi_lag_3,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34
0,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [573]:
data_final

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_2,cpi_lag_3,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34
0,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000
2,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000
3,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000
4,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469428,2023-11-20,64859258.0,6.485926e+07,NaN,6.485926e+07,NaN,NaN,GH85,NaN,NaN,...,113.28,112.07,111.09,112.66,112.66,112.66,112.66,0.000883,0.010797,0.008822
469429,2023-12-04,NaN,NaN,NaN,0.000000e+00,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469430,2023-12-11,NaN,NaN,NaN,0.000000e+00,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469431,2023-12-18,NaN,NaN,NaN,0.000000e+00,NaN,NaN,GH85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Holidays

In [574]:
Holidays_df= pd.DataFrame()

#Holidays list
#2022

List_2021 = [ '2021-01-01','2021-02-10','2021-02-11'
              ,'2021-02-12','2021-02-13','2021-02-14'
             ,'2021-02-15','2021-02-16','2021-04-21'
             ,'2021-04-30','2021-05-01','2021-05-03'
            ,'2021-09-02','2021-09-03','2021-09-04'
             ,'2021-09-05']


List_2022 = [ '2022-01-01','2022-01-02','2022-01-03'
              ,'2022-01-29','2022-01-30','2022-01-31'
             ,'2022-02-01','2022-02-02','2022-02-03'
             ,'2022-02-04','2022-02-05','2022-02-06'
             
             ,'2022-04-10','2022-04-11','2022-04-30'
             ,'2022-05-01','2022-05-02','2022-05-03'
             ,'2022-09-01','2022-09-02']


List_2023 = [ '2023-01-01','2023-01-02'
              ,'2023-01-21','2023-01-22','2023-01-23'
             ,'2023-01-24','2023-01-25','2023-01-26'
             ,'2023-01-27'
             ,'2023-04-29','2023-04-30','2023-05-01'
             ,'2023-05-02'
             ,'2023-09-02','2023-09-04']

Holidays_list = List_2021 + List_2022+List_2023


Holidays_df = pd.DataFrame(Holidays_list, columns=['Holidays_date'])

Holidays_df['Holidays_date']= pd.to_datetime(Holidays_df['Holidays_date'])
Holidays_df['Holidays_monday'] = Holidays_df['Holidays_date'] - pd.to_timedelta(Holidays_df['Holidays_date'].dt.weekday, unit='D')
Holidays_df=Holidays_df.drop(['Holidays_date'], axis=1)


Holidays_df=Holidays_df.groupby(["Holidays_monday"]).size().reset_index(name='Holidays_count')

data_final= data_final.merge(Holidays_df, left_on='CALDAY', right_on= 'Holidays_monday', how ='left' )
data_final=data_final.drop(['Holidays_monday'], axis=1)

data_final['Holidays_count'] = data_final['Holidays_count'].fillna(0)


In [575]:
data_final['Holidays_count'].max()

7.0

In [576]:
data_final['CALDAY'].min()

Timestamp('2020-12-28 00:00:00')

### Data Cleaning

In [577]:
# Remove sku-store combination if data is less than 6 months

sufficient_data = data_final.reset_index().groupby(['mch_store'], as_index=False).size()
sufficient_data
sufficient_data = sufficient_data.loc[sufficient_data['size'] > 24]
sufficient_data_list = sufficient_data['mch_store']
len(sufficient_data_list)

3696

In [578]:
sufficient_data_list

0       1511
2       1513
3       1515
4       1518
5       1519
        ... 
3895    6A18
3896    6A19
3897    6A20
3898    6A21
3899    6A23
Name: mch_store, Length: 3696, dtype: object

In [579]:
#for 1 store there should be 25*96 = 2400 observations

sufficient_data_list = sufficient_data_list.tolist()
data_final = data_final[data_final['mch_store'].isin(sufficient_data_list)]
data_final.shape

(466082, 113)

In [580]:
data_final['agg_sale'].min()

0.0

In [581]:
data_final.head()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_3,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34,Holidays_count
0,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [582]:
#Min date
First_transaction_date = data_final.reset_index().groupby(['mch_store'], as_index=False)['CALDAY'].agg(min)
First_transaction_date.rename(columns = {'CALDAY':'First_date'}, inplace = True)
#First_transaction_date


In [583]:
import pandas as pd
from datetime import date, timedelta

sdate = date(2021,1,1)   # start date
edate = date(2023,12,31)   # end date

calender=[sdate+timedelta(days=x) for x in range((edate-sdate).days)]
calender_v1 = pd.DataFrame(calender,columns = ['CALDAY'])
calender_v1['CALDAY']= pd.to_datetime(calender_v1['CALDAY'])
calender_v1['CALDAY'].dt.dayofweek

filtered_calender = calender_v1.loc[calender_v1['CALDAY'].dt.dayofweek.isin([0])] 
filtered_calender.shape

(156, 1)

In [584]:
filtered_calender

,CALDAY
3,2021-01-04
10,2021-01-11
17,2021-01-18
24,2021-01-25
31,2021-02-01
...,...
1060,2023-11-27
1067,2023-12-04
1074,2023-12-11
1081,2023-12-18


In [585]:
data_final

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_3,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34,Holidays_count
0,2020-12-28,167834344.0,4.341090e+08,1.882846e+08,6.019433e+08,NaN,NaN,1511,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0
2,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0
3,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0
4,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469080,2023-11-27,1159090.0,4.849152e+07,3.346909e+07,4.965062e+07,9822879.0,405.458888,6A23,1.543581e+07,1.034329e+07,...,112.07,111.09,112.66,112.66,112.66,112.66,0.000883,0.010797,0.008822,0.0
469081,2023-12-04,NaN,NaN,NaN,0.000000e+00,19832831.0,-100.000000,6A23,1.571063e+07,1.005285e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
469082,2023-12-11,NaN,NaN,NaN,0.000000e+00,18786261.0,-100.000000,6A23,1.589155e+07,9.762171e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
469083,2023-12-18,NaN,NaN,NaN,0.000000e+00,22943260.0,-100.000000,6A23,1.628331e+07,9.615440e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [586]:
store_list=data_final["mch_store"].unique().tolist()
data_final.dtypes

CALDAY                   datetime64[ns]
min_sale                        float64
max_sale                        float64
std_sale                        float64
agg_sale                        float64
                              ...      
cpi_perc_change_lag4            float64
cpi_perc_change_lag12           float64
cpi_perc_change_lag23           float64
cpi_perc_change_lag34           float64
Holidays_count                  float64
Length: 113, dtype: object

In [587]:
## what is happening here?

In [588]:
clean_data = pd.DataFrame()

for i in range(0,len(store_list)):
    a = store_list[i]
    filtered_v1 = data_final.loc[df1["mch_store"] == a]
    filtered_v2 =  filtered_calender.merge(filtered_v1, on ='CALDAY', how='left')
    filtered_v2[["mch_store"]] = filtered_v2[["mch_store"]].fillna(a)
    
    clean_data = pd.concat([clean_data, filtered_v2])   
    

In [589]:
clean_data.shape

(576576, 113)

In [590]:
clean_data.tail()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_3,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34,Holidays_count
151,2023-11-27,1159090.0,48491525.0,3.346909e+07,49650615.0,9822879.0,405.458888,6A23,1.543581e+07,1.034329e+07,...,112.07,111.09,112.66,112.66,112.66,112.66,0.000883,0.010797,0.008822,0.0
152,2023-12-04,NaN,NaN,NaN,0.0,19832831.0,-100.000000,6A23,1.571063e+07,1.005285e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
153,2023-12-11,NaN,NaN,NaN,0.0,18786261.0,-100.000000,6A23,1.589155e+07,9.762171e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
154,2023-12-18,NaN,NaN,NaN,0.0,22943260.0,-100.000000,6A23,1.628331e+07,9.615440e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
155,2023-12-25,NaN,NaN,NaN,0.0,33444716.0,-100.000000,6A23,1.718654e+07,1.014007e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [591]:
First_transaction_date1= clean_data.merge(First_transaction_date, on='mch_store', how ='left')
First_transaction_date1 = First_transaction_date1[First_transaction_date1['CALDAY'] >= First_transaction_date1['First_date']]
print(First_transaction_date1.shape)

(477229, 114)


In [592]:
First_transaction_date1.tail()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34,Holidays_count,First_date
576571,2023-11-27,1159090.0,48491525.0,3.346909e+07,49650615.0,9822879.0,405.458888,6A23,1.543581e+07,1.034329e+07,...,111.09,112.66,112.66,112.66,112.66,0.000883,0.010797,0.008822,0.0,2023-06-26
576572,2023-12-04,NaN,NaN,NaN,0.0,19832831.0,-100.000000,6A23,1.571063e+07,1.005285e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26
576573,2023-12-11,NaN,NaN,NaN,0.0,18786261.0,-100.000000,6A23,1.589155e+07,9.762171e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26
576574,2023-12-18,NaN,NaN,NaN,0.0,22943260.0,-100.000000,6A23,1.628331e+07,9.615440e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26
576575,2023-12-25,NaN,NaN,NaN,0.0,33444716.0,-100.000000,6A23,1.718654e+07,1.014007e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26


In [593]:
data_final1 = First_transaction_date1.copy()
data_final1= data_final1.sort_values(['mch_store', 'CALDAY'], ascending=[True, True])

# round the numbers up to the nearest integer

data_final1.tail()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34,Holidays_count,First_date
576571,2023-11-27,1159090.0,48491525.0,3.346909e+07,49650615.0,9822879.0,405.458888,6A23,1.543581e+07,1.034329e+07,...,111.09,112.66,112.66,112.66,112.66,0.000883,0.010797,0.008822,0.0,2023-06-26
576572,2023-12-04,NaN,NaN,NaN,0.0,19832831.0,-100.000000,6A23,1.571063e+07,1.005285e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26
576573,2023-12-11,NaN,NaN,NaN,0.0,18786261.0,-100.000000,6A23,1.589155e+07,9.762171e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26
576574,2023-12-18,NaN,NaN,NaN,0.0,22943260.0,-100.000000,6A23,1.628331e+07,9.615440e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26
576575,2023-12-25,NaN,NaN,NaN,0.0,33444716.0,-100.000000,6A23,1.718654e+07,1.014007e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2023-06-26


In [594]:
data_final1[data_final1['mch_store'] == '1518'].tail()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,cpi_lag_4,cpi_perc_change_lag1,cpi_perc_change_lag2,cpi_perc_change_lag3,cpi_perc_change_lag4,cpi_perc_change_lag12,cpi_perc_change_lag23,cpi_perc_change_lag34,Holidays_count,First_date
619,2023-11-27,322655090.0,1.282327e+09,6.785903e+08,1.604982e+09,1.294159e+09,24.017335,1518,1.462263e+09,5.319882e+08,...,111.09,112.66,112.66,112.66,112.66,0.000883,0.010797,0.008822,0.0,2020-12-28
620,2023-12-04,NaN,NaN,NaN,0.000000e+00,1.131523e+09,-100.000000,1518,1.459982e+09,5.308488e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2020-12-28
621,2023-12-11,NaN,NaN,NaN,0.000000e+00,1.623319e+09,-100.000000,1518,1.461100e+09,5.291878e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2020-12-28
622,2023-12-18,NaN,NaN,NaN,0.000000e+00,8.588382e+08,-100.000000,1518,1.457003e+09,5.297066e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2020-12-28
623,2023-12-25,NaN,NaN,NaN,0.000000e+00,1.164680e+09,-100.000000,1518,1.455028e+09,5.284484e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2020-12-28


In [595]:
#data_final1[(data_final1['mch_store'] == 'Đồ uống, thuốc lá_1511') & (data_final1['CALDAY'] == '2023-10-02')]

In [596]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False


def xgb_mape(preds, dtrain):
   labels = dtrain.get_label()
   return('mape', np.mean(np.abs((labels - preds) / (labels+1))))


from sklearn import metrics

def scoring(y_true, y_pred):
    r2 = round(metrics.r2_score(y_true, y_pred) * 100, 3)
    #  mae = round(metrics.mean_absolute_error(y_true, y_pred), 3)
    mape = round(
        metrics.mean_absolute_percentage_error(y_true, y_pred) * 100, 3
    )
    rmse = round(metrics.mean_squared_error(y_true, y_pred, squared=False), 3)
    
    df = pd.DataFrame({
        'R2': r2,
        'RMSE': rmse,
        'MAPE': mape
    }, index=[0])
    
    return df



def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [597]:
#data_final1.fillna(1,inplace=True)
#data_final1.replace([np.inf, -np.inf], 0)


In [598]:
#df.head()
#df[(df['MCH3_DESC'] == 'Đồ uống, thuốc lá') & (df['CALDAY'] == '2023-10-30')].head()

In [599]:
# Date based features
data_final1['year'] = data_final1['CALDAY'].dt.year
data_final1['weekofyear'] = data_final1['CALDAY'].dt.isocalendar().week
data_final1['month'] = data_final1['CALDAY'].dt.month
data_final1['day'] = data_final1['CALDAY'].dt.day
data_final1['dayofyear'] = data_final1['CALDAY'].dt.dayofyear
data_final1['dayofweek'] = data_final1['CALDAY'].dt.dayofweek
data_final1['quarter'] = data_final1['CALDAY'].dt.quarter
data_final1['days_in_month'] = data_final1['CALDAY'].dt.days_in_month
data_final1['is_month_start'] = data_final1['CALDAY'].dt.is_month_start
data_final1['is_month_end'] = data_final1['CALDAY'].dt.is_month_end
data_final1['is_quarter_start'] = data_final1['CALDAY'].dt.is_quarter_start
data_final1['is_quarter_end'] = data_final1['CALDAY'].dt.is_quarter_end
data_final1['is_year_start'] = data_final1['CALDAY'].dt.is_year_start
data_final1['is_year_end'] = data_final1['CALDAY'].dt.is_year_end

In [600]:
data_final1.head(10)

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,dayofyear,dayofweek,quarter,days_in_month,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end
0,2021-01-04,1.885254e+08,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,4,0,1,31,False,False,False,False,False,False
1,2021-01-11,2.469085e+08,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,11,0,1,31,False,False,False,False,False,False
2,2021-01-18,2.320193e+08,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,18,0,1,31,False,False,False,False,False,False
3,2021-01-25,3.155683e+08,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,25,0,1,31,False,False,False,False,False,False
4,2021-02-01,3.411797e+08,1.720520e+09,9.753411e+08,2.061700e+09,NaN,NaN,1511,NaN,NaN,...,32,0,1,28,True,False,False,False,False,False
5,2021-02-08,1.162177e+09,1.162177e+09,NaN,1.162177e+09,NaN,NaN,1511,NaN,NaN,...,39,0,1,28,False,False,False,False,False,False
6,2021-02-15,1.107816e+08,5.064076e+08,2.797498e+08,6.171892e+08,NaN,NaN,1511,NaN,NaN,...,46,0,1,28,False,False,False,False,False,False
7,2021-02-22,1.642332e+08,6.193793e+08,3.218369e+08,7.836125e+08,6.019433e+08,30.180442,1511,6.019433e+08,NaN,...,53,0,1,28,False,False,False,False,False,False
8,2021-03-01,1.833503e+08,6.429289e+08,3.249711e+08,8.262792e+08,1.070781e+09,-22.833961,1511,8.363621e+08,3.315182e+08,...,60,0,1,31,True,False,False,False,False,False
9,2021-03-08,1.676150e+08,6.991126e+08,3.758255e+08,8.667275e+08,1.206850e+09,-28.182692,1511,9.598582e+08,3.173422e+08,...,67,0,1,31,False,False,False,False,False,False


In [601]:
data_final1['agg_sale'].min()

0.0

### Seasonality index

In [602]:
import pandas as pd
import numpy as np

# Calculate average sales for each month, day of the week, and week of the year
monthly_avg_sales = data_final1.groupby('month')['agg_sale'].mean()
weekly_avg_sales = data_final1.groupby('dayofweek')['agg_sale'].mean()
week_of_year_avg_sales = data_final1.groupby('weekofyear')['agg_sale'].mean()

# Create seasonality indexes
data_final1['MonthlySeasonalityIndex'] = data_final1['month'].map(monthly_avg_sales) / data_final1['agg_sale'].mean()
data_final1['WeeklySeasonalityIndex'] = data_final1['dayofweek'].map(weekly_avg_sales) / data_final1['agg_sale'].mean()
data_final1['WeekOfYearSeasonalityIndex'] = data_final1['weekofyear'].map(week_of_year_avg_sales) / data_final1['agg_sale'].mean()

# Display the updated DataFrame with seasonality indexes
data_final1.head()


,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,days_in_month,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,MonthlySeasonalityIndex,WeeklySeasonalityIndex,WeekOfYearSeasonalityIndex
0,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,31,False,False,False,False,False,False,1.177316,1.0,1.034415
1,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,31,False,False,False,False,False,False,1.177316,1.0,1.192373
2,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,31,False,False,False,False,False,False,1.177316,1.0,1.423773
3,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,31,False,False,False,False,False,False,1.177316,1.0,1.422397
4,2021-02-01,341179658.0,1.720520e+09,9.753411e+08,2.061700e+09,NaN,NaN,1511,NaN,NaN,...,28,True,False,False,False,False,False,1.021412,1.0,1.052307


In [603]:

def SI_lag(df):
    for i in range(order,10):
        df.sort_values(['mch_store','CALDAY'],inplace=True)
        df["WeeklySeasonalityIndex" + str(i)] = df.groupby(['mch_store'])['WeeklySeasonalityIndex'].shift(i)
        df["MonthlySeasonalityIndex" + str(i)] = df.groupby(['mch_store'])['MonthlySeasonalityIndex'].shift(i)
        df["WeekOfYearSeasonalityIndex" + str(i)] = df.groupby(['mch_store'])['WeekOfYearSeasonalityIndex'].shift(i)
    return(df)

In [604]:
SI_lag(data_final1)

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,is_year_end,MonthlySeasonalityIndex,WeeklySeasonalityIndex,WeekOfYearSeasonalityIndex,WeeklySeasonalityIndex8,MonthlySeasonalityIndex8,WeekOfYearSeasonalityIndex8,WeeklySeasonalityIndex9,MonthlySeasonalityIndex9,WeekOfYearSeasonalityIndex9
0,2021-01-04,188525402.0,8.822555e+08,4.905413e+08,1.070781e+09,NaN,NaN,1511,NaN,NaN,...,False,1.177316,1.0,1.034415,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-11,246908517.0,9.599420e+08,5.041908e+08,1.206850e+09,NaN,NaN,1511,NaN,NaN,...,False,1.177316,1.0,1.192373,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-18,232019341.0,9.806730e+08,5.293781e+08,1.212692e+09,NaN,NaN,1511,NaN,NaN,...,False,1.177316,1.0,1.423773,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-25,315568256.0,1.624548e+09,9.255883e+08,1.940116e+09,NaN,NaN,1511,NaN,NaN,...,False,1.177316,1.0,1.422397,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-01,341179658.0,1.720520e+09,9.753411e+08,2.061700e+09,NaN,NaN,1511,NaN,NaN,...,False,1.021412,1.0,1.052307,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576571,2023-11-27,1159090.0,4.849152e+07,3.346909e+07,4.965062e+07,9822879.0,405.458888,6A23,1.543581e+07,1.034329e+07,...,False,0.959080,1.0,0.922847,1.0,0.953202,0.993460,1.0,1.043289,1.032256
576572,2023-12-04,NaN,NaN,NaN,0.000000e+00,19832831.0,-100.000000,6A23,1.571063e+07,1.005285e+07,...,False,0.587854,1.0,0.569730,1.0,0.953202,0.966325,1.0,0.953202,0.993460
576573,2023-12-11,NaN,NaN,NaN,0.000000e+00,18786261.0,-100.000000,6A23,1.589155e+07,9.762171e+06,...,False,0.587854,1.0,0.574904,1.0,0.953202,0.946147,1.0,0.953202,0.966325
576574,2023-12-18,NaN,NaN,NaN,0.000000e+00,22943260.0,-100.000000,6A23,1.628331e+07,9.615440e+06,...,False,0.587854,1.0,0.586715,1.0,0.953202,0.935230,1.0,0.953202,0.946147


In [605]:
data_final1.fillna(0,inplace=True)

In [606]:
data_final1['mch_store'].nunique()

3696

In [607]:
data_final1.tail()

,CALDAY,min_sale,max_sale,std_sale,agg_sale,lag_8,sales_growth,mch_store,expanding_mean_0,expanding_std_0,...,is_year_end,MonthlySeasonalityIndex,WeeklySeasonalityIndex,WeekOfYearSeasonalityIndex,WeeklySeasonalityIndex8,MonthlySeasonalityIndex8,WeekOfYearSeasonalityIndex8,WeeklySeasonalityIndex9,MonthlySeasonalityIndex9,WeekOfYearSeasonalityIndex9
576571,2023-11-27,1159090.0,48491525.0,3.346909e+07,49650615.0,9822879.0,405.458888,6A23,1.543581e+07,1.034329e+07,...,False,0.959080,1.0,0.922847,1.0,0.953202,0.993460,1.0,1.043289,1.032256
576572,2023-12-04,0.0,0.0,0.000000e+00,0.0,19832831.0,-100.000000,6A23,1.571063e+07,1.005285e+07,...,False,0.587854,1.0,0.569730,1.0,0.953202,0.966325,1.0,0.953202,0.993460
576573,2023-12-11,0.0,0.0,0.000000e+00,0.0,18786261.0,-100.000000,6A23,1.589155e+07,9.762171e+06,...,False,0.587854,1.0,0.574904,1.0,0.953202,0.946147,1.0,0.953202,0.966325
576574,2023-12-18,0.0,0.0,0.000000e+00,0.0,22943260.0,-100.000000,6A23,1.628331e+07,9.615440e+06,...,False,0.587854,1.0,0.586715,1.0,0.953202,0.935230,1.0,0.953202,0.946147
576575,2023-12-25,0.0,0.0,0.000000e+00,0.0,33444716.0,-100.000000,6A23,1.718654e+07,1.014007e+07,...,False,0.587854,1.0,0.619466,1.0,0.953202,0.974462,1.0,0.953202,0.935230


In [608]:
### change the split

In [609]:
date_split1 = "2023-01-01"
date_split = "2023-09-01"

In [610]:
train = data_final1[data_final1['CALDAY'] <= date_split1  ]
test = data_final1[data_final1['CALDAY'] >= date_split  ]
test = test[test['mch_store'].isin(train['mch_store'])]
train['mch_store'].nunique()


3523

In [611]:
test.CALDAY.max()

Timestamp('2023-12-25 00:00:00')

In [612]:
eval_ = data_final1[(data_final1['CALDAY'] >= date_split1) & (data_final1['CALDAY'] <= date_split)  ]
eval_ = eval_[eval_['mch_store'].isin(test['mch_store'])]
eval_.dropna(subset=['agg_sale'],inplace=True)
eval_['mch_store'].nunique()


3523

In [613]:
train = data_final1[data_final1['CALDAY'] <= date_split1  ]
train = train[train['mch_store'].isin(test['mch_store'])]
train.dropna(subset=['agg_sale'],inplace=True)
train['mch_store'].nunique()

3523

In [614]:
data_final1 = pd.concat([train, eval_, test], ignore_index=True)
data_final1.shape

(470887, 137)

## SKIP TO LOWER CODE

In [615]:
data_final1.dropna(subset=['agg_sale'],inplace=True)

In [616]:
list(data_final1.columns)

['CALDAY',
 'min_sale',
 'max_sale',
 'std_sale',
 'agg_sale',
 'lag_8',
 'sales_growth',
 'mch_store',
 'expanding_mean_0',
 'expanding_std_0',
 'expanding_min_0',
 'expanding_max_0',
 'expanding_mean_1',
 'expanding_std_1',
 'expanding_min_1',
 'expanding_max_1',
 'expanding_mean_2',
 'expanding_std_2',
 'expanding_min_2',
 'expanding_max_2',
 'expanding_mean_3',
 'expanding_std_3',
 'expanding_min_3',
 'expanding_max_3',
 'expanding_mean_4',
 'expanding_std_4',
 'expanding_min_4',
 'expanding_max_4',
 'expanding_mean_5',
 'expanding_std_5',
 'expanding_min_5',
 'expanding_max_5',
 'expanding_mean_6',
 'expanding_std_6',
 'expanding_min_6',
 'expanding_max_6',
 'expanding_mean_7',
 'expanding_std_7',
 'expanding_min_7',
 'expanding_max_7',
 'ewma_mean_2',
 'ewma_std_2',
 'ewma_mean_3',
 'ewma_std_3',
 'ewma_mean_4',
 'ewma_std_4',
 'ewma_mean_5',
 'ewma_std_5',
 'ewma_mean_6',
 'ewma_std_6',
 'ewma_mean_7',
 'ewma_std_7',
 'ewma_mean_8',
 'ewma_std_8',
 'ewma_mean_9',
 'ewma_std_9',


In [617]:
data_final1['CALDAY'].max()

Timestamp('2023-12-25 00:00:00')

In [353]:
brand_list=data_final1["mch_store"].unique().tolist()
#brand_list = brand_list[:10]
brand_list = ['1518']
len(brand_list)

1

## Tune parameters

In [354]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import StratifiedKFold
from skopt.callbacks import DeltaYStopper, DeadlineStopper
from time import time
import pprint

import warnings



mape_df = pd.DataFrame()
R2_df = pd.DataFrame()
forecast_df = pd.DataFrame()
final_features_top = []
count = 0

date_split1 = "2023-01-01"
date_split = "2023-09-01"

start = time()

for i in range(0, len(brand_list)):
    count = count + 1
    a = brand_list[i]
    print(a)
    data_final_v1 = data_final1.loc[data_final1["mch_store"] == a]
    data_final_v1 = data_final_v1.sort_values(['mch_store', 'CALDAY'], ascending=[True, True])

    week_nbr = data_final_v1['CALDAY']
    sale_amt = data_final_v1['agg_sale']

    train = data_final_v1.loc[data_final_v1['CALDAY'] < date_split1]
    eval_ = data_final_v1.loc[(data_final_v1['CALDAY'] >= date_split1) & (data_final_v1['CALDAY'] < date_split)]
    test = data_final_v1.loc[data_final_v1['CALDAY'] >= date_split]

    cols1 = ["CALDAY", "mch_store", "First_date"]

    train = train.drop(cols1, axis=1)
    test = test.drop(cols1, axis=1)
    eval_ = eval_.drop(cols1, axis=1)

    X_train = train[train.columns[train.columns != 'agg_sale']]
    y_train = train['agg_sale']

    X_test = test[test.columns[test.columns != 'agg_sale']]
#     y_test = test['agg_sale']

    X_eval = eval_[eval_.columns[eval_.columns != 'agg_sale']]
    y_eval = eval_['agg_sale']
    
    param_grid = {

    'n_estimators': [300, 500],
    'max_depth': [2,8, 16],
    'learning_rate': [0.01, 0.05, 0.1],
    'boosting_type': ['gbdt'],
    'objective': ['regression_l1'],   #no mape
    'metric': ['rmse'],
    'num_leaves': [2, 8, 16],
    'num_iteration': [500],
    'reg_lambda': [0.01, 0.1, 0.5],
    'reg_alpha': [0.01, 0.1, 0.5],
    'min_child_samples': [1],  #change this next
 #  'feature_fraction': [0.5, 0.8, 1.0],   
    'early_stopping': [None],
         'verbose': [-1 ]

    }
    param_candidates = ParameterGrid(param_grid)
    print(f'{len(param_candidates)} candidates')

    lgb_regressor = lgb.LGBMRegressor(boosting_type='gbdt',
                                      objective='regression_l1',
                                      metric='mape',
                                      n_jobs=1,
                                      verbose=-1,
                                      random_state=0)

    def fit_model(params):
            warnings.filterwarnings("ignore")
            model1 = lgb_regressor.set_params(**params)
            model1.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_eval, y_eval)])
            score = model1.score(X_eval, y_eval)
            return [params, score]
        
    warnings.filterwarnings("ignore")
    results = Parallel(n_jobs=-1, verbose=0)(delayed(fit_model)(params) for params in param_candidates)
    print(max(results, key=lambda x: x[1]))

    final_params = max(results, key=lambda x: x[1])[0]
    R2 = max(results, key=lambda x: x[1])[1]

    model = lgb_regressor.set_params(**final_params)
    warnings.filterwarnings("ignore")
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_eval, y_eval)])

    f_i = list(zip(X_train.columns.values, model.feature_importances_))
    f_i.sort(key=lambda x: x[1])

    final_features = [i[0] for i in f_i][-20:]
    final_features_top = final_features_top + final_features

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_pred_eval = model.predict(X_eval)
        

    b = np.array([a] * (len(y_pred_train) + len(y_pred_eval) + len(y_pred_test) ))
    preds = np.append(y_pred_train, np.append(y_pred_eval, y_pred_test))

    forecast = pd.concat({'mch_store': pd.DataFrame(b), 'week_nbr': pd.DataFrame(week_nbr).reset_index(drop=True),
                              'sale_amt': pd.DataFrame(sale_amt).reset_index(drop=True),
                              'prediction': pd.DataFrame(preds)}, axis=1)
    forecast_df = pd.concat([forecast_df, forecast])

    mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
    mape_eval = mean_absolute_percentage_error(y_eval, y_pred_eval)
    #mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

    temp = pd.DataFrame({'mch_store': a, 'mape_train': mape_train, 'mape_eval': mape_eval}, index=[0])
    temp1 = pd.DataFrame({'mch_store': a, 'R-Square': R2, 'params': [final_params], 'top_features': [final_features]},
                             index=[0])

    mape_df = pd.concat([mape_df, temp])
    R2_df = pd.concat([R2_df, temp1])
end = time()
print(f"Execution time: {end - start} seconds")

1518
486 candidates
[{'boosting_type': 'gbdt', 'early_stopping': None, 'learning_rate': 0.05, 'max_depth': 8, 'metric': 'rmse', 'min_child_samples': 1, 'n_estimators': 300, 'num_iteration': 500, 'num_leaves': 8, 'objective': 'regression_l1', 'reg_alpha': 0.01, 'reg_lambda': 0.5, 'verbose': -1}, 0.9881059449194466]
Execution time: 29.467285871505737 seconds


In [355]:
model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.05,
 'max_depth': 8,
 'min_child_samples': 1,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 300,
 'n_jobs': 1,
 'num_leaves': 8,
 'objective': 'regression_l1',
 'random_state': 0,
 'reg_alpha': 0.01,
 'reg_lambda': 0.5,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'metric': 'rmse',
 'verbose': -1,
 'early_stopping': None,
 'num_iteration': 500}

In [356]:
forecast_df

,mch_store,week_nbr,sale_amt,prediction
,0,CALDAY,agg_sale,0
0,1518,2021-01-04,1.568104e+09,1.573962e+09
1,1518,2021-01-11,1.814816e+09,1.816380e+09
2,1518,2021-01-18,1.751977e+09,1.752379e+09
3,1518,2021-01-25,2.237087e+09,2.237053e+09
4,1518,2021-02-01,3.300130e+09,3.300215e+09
...,...,...,...,...
151,1518,2023-11-27,0.000000e+00,4.801766e+08
152,1518,2023-12-04,0.000000e+00,5.008760e+08
153,1518,2023-12-11,0.000000e+00,4.865155e+08


In [357]:
print(mape_df["mape_train"].mean())
print(mape_df["mape_eval"].mean())


inf
2.886188187894327


In [ ]:
train_forecast = forecast_df[(forecast_df['week_nbr']['CALDAY'] < date_split1) ]
train_forecast.to_csv('inf.csv')
test_forecast = forecast_df[(forecast_df['week_nbr']['CALDAY'] >= date_split) ]
test_forecast.reset_index(inplace=True,drop=True)
test_forecast.to_csv("future_forecast.csv")
test_df = data_final1[data_final1['mch_store'] == '1518']
train = test_df.loc[test_df['CALDAY'] < date_split1]
eval_ = test_df.loc[(test_df['CALDAY'] >= date_split1) & (test_df['CALDAY'] < date_split)]
test = test_df.loc[test_df['CALDAY'] >= date_split]
#test_forecast



### Run the model for 10 of the top 100 underperforming stores

In [782]:
stores = pd.read_csv("mapetest3000.csv")
stores = stores.iloc[1:, 1:]
stores.to_csv('your_output_file.csv', index=False, float_format='%.6f')
stores =  pd.read_csv("your_output_file.csv")
stores['week_nbr'] = pd.to_datetime(stores['week_nbr'])
stores['year_month'] = stores['week_nbr'].dt.to_period('M')
stores.head()



,mch_store,week_nbr,sale_amt,prediction,year_month
0,1511,2023-01-02,1.059926e+09,1.110064e+09,2023-01
1,1511,2023-01-09,1.252613e+09,1.247864e+09,2023-01
2,1511,2023-01-16,1.333787e+09,1.435014e+09,2023-01
3,1511,2023-01-23,4.375291e+08,5.054621e+08,2023-01
4,1511,2023-01-30,7.691016e+08,8.218420e+08,2023-01


In [783]:
stores.shape

(137627, 5)

In [784]:
stores = stores[stores['sale_amt'] != 0]
stores.shape

(137627, 5)

In [785]:
monthly_df = stores.groupby(['mch_store', 'year_month']).agg({'sale_amt': 'sum', 'prediction': 'sum'}).reset_index()
monthly_df['year_month'] = monthly_df['year_month'].dt.to_timestamp()

monthly_df['absolute_percentage_error'] = abs((monthly_df['sale_amt'] - monthly_df['prediction']) / monthly_df['sale_amt'])
mape = monthly_df['absolute_percentage_error'].mean()

print(f'MAPE: {mape * 100:.2f}%')
monthly_df.head()


MAPE: 8.91%


,mch_store,year_month,sale_amt,prediction,absolute_percentage_error
0,1511,2023-01-01,4.852955e+09,5.120247e+09,0.055078
1,1511,2023-02-01,3.083297e+09,3.178336e+09,0.030824
2,1511,2023-03-01,2.845051e+09,2.810654e+09,0.012090
3,1511,2023-04-01,2.814026e+09,2.901203e+09,0.030979
4,1511,2023-05-01,3.563076e+09,3.835820e+09,0.076547


In [734]:
#mape_by_store[mape_by_store['mch_store'] == '3888']
df[df['STORE_ID'] == '3888'].calday.max()

Timestamp('2023-11-27 00:00:00')

In [739]:
#stores[stores['mch_store'] == '3888']


,mch_store,week_nbr,sale_amt,prediction,year_month


In [741]:

stores = stores[stores['week_nbr'] > '2023-10-01']
stores['mch_store'].nunique()  #2905

monthly_df = stores.groupby(['mch_store', 'year_month']).agg({'sale_amt': 'sum', 'prediction': 'sum'}).reset_index()
monthly_df['year_month'] = monthly_df['year_month'].dt.to_timestamp()      
monthly_df['absolute_percentage_error'] = abs((monthly_df['sale_amt'] - monthly_df['prediction']) / monthly_df['sale_amt'])

mape_by_store = monthly_df.groupby('mch_store')['absolute_percentage_error'].mean().reset_index()
mape_by_store

,mch_store,absolute_percentage_error
0,1513,0.009408
1,1515,0.010149
2,1518,0.021636
3,1519,0.061152
4,1524,0.039499
...,...,...
2900,6480,0.185258
2901,6482,0.469831
2902,6485,1.559614
2903,6492,0.269956


In [742]:
mape_by_store[mape_by_store['mch_store'] == '3888']

,mch_store,absolute_percentage_error


#### Mape for just for the month of Oct

In [737]:
mape_by_store = monthly_df[monthly_df['year_month'] == '2023-10-01']
mape_by_store = mape_by_store.groupby('mch_store')['absolute_percentage_error'].mean().reset_index()
mape_by_store['absolute_percentage_error'].mean()

0.08060516081749575

In [743]:
mape_by_store = mape_by_store.sort_values(by='absolute_percentage_error', ascending=True)
top10 = mape_by_store[:10]
top10['flag'] = 'top10'
last10 = mape_by_store.tail(10)
last10['flag'] = 'last10'

concatenated_df = pd.concat([top10, last10], ignore_index=True)
concatenated_df.head()

,mch_store,absolute_percentage_error,flag
0,4668,0.000486,top10
1,5637,0.000602,top10
2,3528,0.000681,top10
3,5151,0.000839,top10
4,3347,0.000843,top10


In [744]:
concatenated_df['mch_store'] = concatenated_df['mch_store'].astype(str)
brand_list2 = concatenated_df["mch_store"].unique().tolist()
len(brand_list2)

20

In [745]:
brand_list2

['4668',
 '5637',
 '3528',
 '5151',
 '3347',
 '4967',
 '2210',
 '5426',
 '3948',
 '3824',
 '1663',
 '6381',
 '3419',
 '1703',
 '6444',
 '5859',
 '6485',
 '5998',
 '4117',
 '1704']

In [746]:
date_split1 = "2023-01-01"
date_split = "2023-09-01"

# train = data_final_v1[data_final_v1['CALDAY'] < date_split1]
# eval_ = data_final_v1[(data_final_v1['CALDAY'] >= date_split1) & (data_final_v1['CALDAY'] < date_split)]
# test = data_final_v1[data_final_v1['CALDAY'] >= date_split]

In [747]:
concatenated_df['mch_store'].dtypes

dtype('O')

In [748]:
concatenated_df.tail()

,mch_store,absolute_percentage_error,flag
15,5859,1.499161,last10
16,6485,1.559614,last10
17,5998,1.659665,last10
18,4117,1.844093,last10
19,1704,2.598132,last10


In [749]:
#eval_

#### Run future forecasts for these top10 and last 10  stores

In [750]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import StratifiedKFold
from skopt.callbacks import DeltaYStopper, DeadlineStopper
from time import time
import pprint

import warnings


mape_df = pd.DataFrame()
R2_df = pd.DataFrame()
forecast_df = pd.DataFrame()
final_features_top = []
count = 0

date_split1 = "2023-01-01"
date_split = "2023-09-01"

start = time()

for i in range(0, len(brand_list2)):
    #count = count + 1
    a = brand_list2[i]
    print(a)
    
    data_final_v1 = data_final1[data_final1['mch_store'] == a]
    data_final_v1 = data_final_v1.sort_values(['mch_store', 'CALDAY'], ascending=[True, True])
    #print(data_final_v1)

    week_nbr = data_final_v1['CALDAY']
    sale_amt = data_final_v1['agg_sale']

    train = data_final_v1.loc[data_final_v1['CALDAY'] < date_split1]
    eval_ = data_final_v1.loc[(data_final_v1['CALDAY'] >= date_split1) & (data_final_v1['CALDAY'] < date_split)]
    test = data_final_v1.loc[data_final_v1['CALDAY'] >= date_split]

    cols1 = ["CALDAY", "mch_store", "First_date"]
    #print(train)

    train = train.drop(cols1, axis=1)
    test = test.drop(cols1, axis=1)
    eval_ = eval_.drop(cols1, axis=1)

    X_train = train[train.columns[train.columns != 'agg_sale']]
    y_train = train['agg_sale']

    X_test = test[test.columns[test.columns != 'agg_sale']]
#     y_test = test['agg_sale']

    X_eval = eval_[eval_.columns[eval_.columns != 'agg_sale']]
    y_eval = eval_['agg_sale']
    
    param_grid = {

    'n_estimators': [300, 500],
    'max_depth': [2,8, 16],
    'learning_rate': [0.01, 0.05, 0.1],
    'boosting_type': ['gbdt'],
    'objective': ['regression_l1'],   #no mape
    'metric': ['rmse'],
    'num_leaves': [2, 8, 16],
    'num_iteration': [500],
    'reg_lambda': [0.01, 0.1, 0.5],
    'reg_alpha': [0.01, 0.1, 0.5],
    'min_child_samples': [1],  #change this next
 #  'feature_fraction': [0.5, 0.8, 1.0],   
    'early_stopping': [None],
         'verbose': [-1 ]

    }
    param_candidates = ParameterGrid(param_grid)
    print(f'{len(param_candidates)} candidates')

    lgb_regressor = lgb.LGBMRegressor(boosting_type='gbdt',
                                      objective='regression_l1',
                                      metric='mape',
                                      n_jobs=1,
                                      verbose=-1,
                                      random_state=0)

    def fit_model(params):
            warnings.filterwarnings("ignore")
            model1 = lgb_regressor.set_params(**params)
            model1.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_eval, y_eval)])
            score = model1.score(X_eval, y_eval)
            return [params, score]
        
    warnings.filterwarnings("ignore")
    results = Parallel(n_jobs=-1, verbose=0)(delayed(fit_model)(params) for params in param_candidates)
    print(max(results, key=lambda x: x[1]))

    final_params = max(results, key=lambda x: x[1])[0]
    R2 = max(results, key=lambda x: x[1])[1]

    model = lgb_regressor.set_params(**final_params)
    warnings.filterwarnings("ignore")
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_eval, y_eval)])

    f_i = list(zip(X_train.columns.values, model.feature_importances_))
    f_i.sort(key=lambda x: x[1])

    final_features = [i[0] for i in f_i][-20:]
    final_features_top = final_features_top + final_features

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_pred_eval = model.predict(X_eval)
        

    b = np.array([a] * (len(y_pred_train) + len(y_pred_eval) + len(y_pred_test) ))
    preds = np.append(y_pred_train, np.append(y_pred_eval, y_pred_test))

    forecast = pd.concat({'mch_store': pd.DataFrame(b), 'week_nbr': pd.DataFrame(week_nbr).reset_index(drop=True),
                              'sale_amt': pd.DataFrame(sale_amt).reset_index(drop=True),
                              'prediction': pd.DataFrame(preds)}, axis=1)
    forecast_df = pd.concat([forecast_df, forecast])

    mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
    mape_eval = mean_absolute_percentage_error(y_eval, y_pred_eval)
    #mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

    temp = pd.DataFrame({'mch_store': a, 'mape_train': mape_train, 'mape_eval': mape_eval}, index=[0])
    temp1 = pd.DataFrame({'mch_store': a, 'R-Square': R2, 'params': [final_params], 'top_features': [final_features]},
                             index=[0])

    mape_df = pd.concat([mape_df, temp])
    R2_df = pd.concat([R2_df, temp1])
end = time()
print(f"Execution time: {end - start} seconds")

4668
486 candidates
[{'boosting_type': 'gbdt', 'early_stopping': None, 'learning_rate': 0.1, 'max_depth': 8, 'metric': 'rmse', 'min_child_samples': 1, 'n_estimators': 300, 'num_iteration': 500, 'num_leaves': 16, 'objective': 'regression_l1', 'reg_alpha': 0.5, 'reg_lambda': 0.01, 'verbose': -1}, 0.9625608717502362]
5637
486 candidates
[{'boosting_type': 'gbdt', 'early_stopping': None, 'learning_rate': 0.1, 'max_depth': 8, 'metric': 'rmse', 'min_child_samples': 1, 'n_estimators': 300, 'num_iteration': 500, 'num_leaves': 8, 'objective': 'regression_l1', 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'verbose': -1}, 0.5609243053475239]
3528
486 candidates
[{'boosting_type': 'gbdt', 'early_stopping': None, 'learning_rate': 0.1, 'max_depth': 8, 'metric': 'rmse', 'min_child_samples': 1, 'n_estimators': 300, 'num_iteration': 500, 'num_leaves': 8, 'objective': 'regression_l1', 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'verbose': -1}, 0.9449610303993844]
5151
486 candidates
[{'boosting_type': 'gbdt', 'early_

In [764]:
test_forecast = forecast_df[(forecast_df['week_nbr']['CALDAY'] >= date_split) ]
test_forecast.reset_index(inplace=True,drop=True)


In [765]:
test_forecast.columns = [f'{col[0]}_{col[1]}' if isinstance(col[1], int) else col[0] for col in test_forecast.columns]
test_forecast

,mch_store_0,week_nbr,sale_amt,prediction_0
0,4668,2023-09-04,138657246.0,1.364120e+08
1,4668,2023-09-11,144917846.0,1.491711e+08
2,4668,2023-09-18,149533151.0,1.551539e+08
3,4668,2023-09-25,146313737.0,1.505070e+08
4,4668,2023-10-02,167267981.0,1.620755e+08
...,...,...,...,...
335,1704,2023-11-27,902526247.0,5.020204e+08
336,1704,2023-12-04,0.0,2.880120e+08
337,1704,2023-12-11,0.0,2.878371e+08
338,1704,2023-12-18,0.0,2.878371e+08


In [767]:
column_name_mapping = {
    'mch_store_0': 'mch_store',
    'prediction_0': 'predicted_sales',
    'sale_amt' : 'actual_sales',
     'week_nbr' : 'week_start'
}

# Rename the specified columns
test_forecast = test_forecast.rename(columns=column_name_mapping)
test_forecast

,mch_store,week_start,actual_sales,predicted_sales
0,4668,2023-09-04,138657246.0,1.364120e+08
1,4668,2023-09-11,144917846.0,1.491711e+08
2,4668,2023-09-18,149533151.0,1.551539e+08
3,4668,2023-09-25,146313737.0,1.505070e+08
4,4668,2023-10-02,167267981.0,1.620755e+08
...,...,...,...,...
335,1704,2023-11-27,902526247.0,5.020204e+08
336,1704,2023-12-04,0.0,2.880120e+08
337,1704,2023-12-11,0.0,2.878371e+08
338,1704,2023-12-18,0.0,2.878371e+08


In [776]:
result_df = pd.merge(test_forecast ,concatenated_df,on='mch_store', how='inner')
result_df = result_df.rename(columns={'absolute_percentage_error': 'mape_during_train'})
result_df['mape_during_train'] = result_df['mape_during_train'] * 100
#result_df['absolute_percentage_error'] = (abs((result_df['actual_sales'] - result_df['predicted_sales']) / result_df['actual_sales']))*100
result_df

,mch_store,week_start,actual_sales,predicted_sales,mape_during_train,flag
0,4668,2023-09-04,138657246.0,1.364120e+08,0.048581,top10
1,4668,2023-09-11,144917846.0,1.491711e+08,0.048581,top10
2,4668,2023-09-18,149533151.0,1.551539e+08,0.048581,top10
3,4668,2023-09-25,146313737.0,1.505070e+08,0.048581,top10
4,4668,2023-10-02,167267981.0,1.620755e+08,0.048581,top10
...,...,...,...,...,...,...
335,1704,2023-11-27,902526247.0,5.020204e+08,259.813171,last10
336,1704,2023-12-04,0.0,2.880120e+08,259.813171,last10
337,1704,2023-12-11,0.0,2.878371e+08,259.813171,last10
338,1704,2023-12-18,0.0,2.878371e+08,259.813171,last10


In [778]:
result_df.to_csv("Future_Forecast_top10_bottom10.csv",index=False)

In [781]:
chk = data_final1.groupby(['year', 'month']).sum().reset_index()
chk

TypeError: datetime64 type does not support sum operations